In [2]:
import spacy
import json
from spacy.tokens import DocBin
from spacy.lang.tr import Turkish
from tqdm import tqdm
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokenizer import Tokenizer

In [6]:
f = open('./data/TCDD_annotations.json', encoding='utf-8')
data = json.load(f)

In [307]:
nlp = Turkish()
nlp = spacy.blank('tr')

In [315]:
tokenizer = Tokenizer(nlp.vocab)

In [321]:
tokens = tokenizer('5 ocak ankara treni 2 öğretim görevlisi')

In [326]:
type(tokens[0])

spacy.tokens.token.Token

In [4]:
doc_bin = DocBin()

In [5]:
for example in tqdm(data['annotations']):
    text = example[0]
    labels = example[1]['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 678.34it/s]

Skipping entity
Skipping entity


In [8]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-10-28 12:14:28.686948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-28 12:14:28.686996: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-28 12:14:33.601665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-28 12:14:33.601741: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-28 12:14:33.605946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-VR93MKND
2022-10-28 12:14:33.606055: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-VR93MKND


In [9]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     56.43    3.96   10.53    2.44    0.04
[+] Saved pipeline to output directory
model-last


2022-10-28 12:14:56.372874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-28 12:14:56.372968: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-28 12:15:01.202508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-28 12:15:01.202551: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-28 12:15:01.208175: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-VR93MKND
2022-10-28 12:15:01.208381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-VR93MKND
[2022-10-28 12:15:02,152] [INFO] Set up nlp object from config
[2022-10-28 12:15:02,168] [INFO] Pipeline: 

In [2]:
nlp_ner = spacy.load('model-best')

In [3]:
doc = nlp_ner("Seyahat Süresi : Ankara-İstanbul(Söğütlüçeşme) seyahat süresi duruş sayısına göre değişkenlik göstermekle birlikte 4.30- 5 saat aralığındadır..")

colors = {"DURAK": "#F67DE3", "HAT": "#7DF6D9", "SÜRE":"#a6e22d"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [46]:
import random
import re
import pickle

In [45]:
kalkış_list = ['Ankara', 'İstanbul', 'Eskişehir', 'İzmir', 'Kars', 'Konya', 'Malatya', 'Adana']
varış_list = ['Ankara', 'İstanbul', 'Eskişehir', 'İzmir', 'Kars', 'Konya', 'Malatya', 'Adana']
ay_list = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
weekday_list = ['pazartesi', 'salı', 'çarşamba', 'perşembe', 'cuma', 'cumartesi', 'pazar']
yolcu_list = ['yolcu', 'yetişkin', 'kişi', 'çocuk', 'hayvan', 'öğrenci', 'genç', 'öğretmen', 'öğretim görevlisi', 'basın', 'muhabir', 'asker', 'personel', 'çalışan', '60 yaş', '65 yaş']

In [6]:
def tagging(word):
    
    if word in kalkış_list or varış_list: tag = 'DURAK'
    if word in ay_list: tag = 'AY'
    if word.isdigit() == True: tag = 'SAYI'
    if word in yolcu_list: tag = 'YOLCU'
    if word in weekday_list: tag = 'GÜN'
    
    return tag

In [57]:
examples = []
count=0
while (count<5000):
    
    kalkış = random.choice(kalkış_list)
    varış = random.choice(varış_list)
    ay = random.choice(ay_list)   
    gün = random.randrange(1,32)
    weekday = random.choice(weekday_list)
    yolcu = random.choice(yolcu_list)
    sayı = random.randrange(1,5)
    
    variables_list = [kalkış, varış, str(gün), ay, weekday, yolcu, str(sayı)]
    
    sentence = [f'{kalkış} kalkışlı {varış} varışlı {gün} {ay} treni',
                f'{gün} {ay} {kalkış} {varış} treni',
                f'{gün} {ay} {varış} treni {sayı} {yolcu}',
                f'{kalkış}-{varış} {gün} {ay}',
                f'{weekday} kalkan {varış} trenleri',
                f'{weekday} kalkıp {weekday} varan tren',
                f'{varış} {gün} {ay}',
                f'{sayı} {yolcu} {kalkış}-{varış} trenleri',
                f'{gün} {ay} {varış} treni {sayı} {yolcu}',
                f'{weekday} kalkış {gün} {ay} dönüş {varış} trenleri',
               ]
    
    selected_sentence = random.choice(sentence)
    annotations = []
    for variable in variables_list:
        if variable in selected_sentence:
            start = selected_sentence.find(variable)
            end = start + len(variable)
            annotation = (start, end, tagging(variable))
            annotations.append(annotation)
    example = (selected_sentence, annotations)
    if example not in examples and str(sayı) not in str(gün) and kalkış != varış: 
        examples.append(example)
        count+=1

In [48]:
with open('annotations', 'wb') as fp:
    pickle.dump(examples, fp)

In [52]:
with open('annotations', 'rb') as fp:
    annotations = pickle.load(fp)

In [55]:
annotations

[('pazar kalkan Adana trenleri', [(13, 18, 'DURAK'), (0, 5, 'GÜN')]),
 ('Eskişehir kalkışlı Konya varışlı 6 Kasım treni',
  [(0, 9, 'DURAK'), (19, 24, 'DURAK'), (33, 34, 'SAYI'), (35, 40, 'AY')]),
 ('Kars 14 Mart', [(0, 4, 'DURAK'), (5, 7, 'SAYI'), (8, 12, 'AY')]),
 ('salı kalkan Eskişehir trenleri', [(12, 21, 'DURAK'), (0, 4, 'GÜN')]),
 ('Ankara kalkışlı Adana varışlı 17 Nisan treni',
  [(0, 6, 'DURAK'), (16, 21, 'DURAK'), (30, 32, 'SAYI'), (33, 38, 'AY')]),
 ('Malatya kalkışlı Malatya varışlı 2 Nisan treni',
  [(0, 7, 'DURAK'), (0, 7, 'DURAK'), (33, 34, 'SAYI'), (35, 40, 'AY')]),
 ('21 Ekim İzmir treni 4 hayvan',
  [(8, 13, 'DURAK'),
   (0, 2, 'SAYI'),
   (3, 7, 'AY'),
   (22, 28, 'YOLCU'),
   (20, 21, 'SAYI')]),
 ('İstanbul-Ankara 20 Haziran',
  [(0, 8, 'DURAK'), (9, 15, 'DURAK'), (16, 18, 'SAYI'), (19, 26, 'AY')]),
 ('17 Temmuz Eskişehir treni 2 yolcu',
  [(10, 19, 'DURAK'),
   (0, 2, 'SAYI'),
   (3, 9, 'AY'),
   (28, 33, 'YOLCU'),
   (26, 27, 'SAYI')]),
 ('4 genç Ankara-İzmir tren

In [194]:
for match in re.finditer(kalkış, selected_sentence):kalkış_annotation = [(match.start(), match.end(), 'DURAK')]
for match in re.finditer(varış, selected_sentence):varış_annotation = [(match.start(), match.end(), 'DURAK')]
for match in re.finditer(str(gün), selected_sentence):gün_annotation = [(match.start(), match.end(), 'GÜN')]

In [196]:
example = (selected_sentence, kalkış_annotation, varış_annotation, gün_annotation)
print(example)

('16 Ocak İzmir Ankara treni', [(8, 13, 'DURAK')], [(14, 20, 'DURAK')], [(0, 2, 'GÜN')])
